```{index} single: DataFrame; .upd() modelflow extension to pandas
```

# Modelflow extensions to pandas

Any class can have both properties (data) and methods (functions that operate on the data of the particular instance of the class). With object-oriented programming languages like python, classes can be built as supersets of existing classes. The `modelflow` class ```model``` inherits or encapsulates all of the features of the pandas dataframe and extends it in many important ways.  Some of the methods below are standard pandas methods, others have been added to it by `modelflow` features

Data in a `dataframe` can be modified directly with built-in pandas functionalities like `.loc[]` and `eval()`, but `modelflow` extends these capabilities in important ways with the `.upd()` and `.mfcalc()` methods. 

## Setting up the python environment

In order to use ```.upd()``` all of the necessary libraries must be **imported** into the python session.

In [22]:
# HIDDEN in jupyterbook 
#This is code to manage dependencies if the notebook is executed in the google colab cloud service
if 'google.colab' in str(get_ipython()):
  import os
  os.system('apt -qqq install graphviz')
  os.system('pip -qqq install ModelFlowIb ipysheet  --no-dependencies ')
  incolab = True  
else:
  incolab = False 



In [23]:
#Jupyter "magic" commands to ensure notebook displays as expected
%load_ext autoreload 
%autoreload 2

# First import pandas and the model into the  workspace
# There is no problem importing multiple times, though it is not very efficient.
import pandas as pd

from modelclass import model 
# functions that improve rendering of modelflow outputs under Jupyter Notebook
model.widescreen()
model.scroll_off()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


```{index} single: DataFrame; Modelflow specific features
```

## Column names in  Modelflow 
```{margin} Modelflow variable names
Modelflow places more restrictions on column names than do pandas *per se*.

```
While pandas `dataframes` are very liberal in what names can be given to columns, ```modelflow``` is more restrictive.

Specifically, in modelflow a variable name must:

* start with a letter
* be upper case

Thus while all these are legal column names in pandas, some are illegal in modelflow.

```{index} single: DataFrame; Modelflow naming conventions
```
```{index} single: Modelflow; DataFrame naming conventions
```


| Variable Name | Legal in</br> modelfow? | Reason |
|:-------|:-------------|:--------|
| IB | Yes | <span style='color:Green'>Starts with a letter and is uppercase</span> |
| ib | No |<span style='color:red'> lowercase letters are not allowed</span>|
| 42ANSWER | No |<span style='color:Red'> does not start with a letter </span>|
| \_HORSE1 | No |<span style='color:Red'>does not start with a letter </span>|
| A_VERY_LONG_NAME_THAT_IS_LEGAL_3 | Yes |<span style='color:Green'> Starts with a letter and is uppercase </span>|

```{index} single: Modelflow; time index
```
```{index} single: DataFrame; Modelflow time index
```


## .index and time dimensions in Modelflow
As we saw above, series have indices.  Dataframes also have indices, which are the row names of the dataframe.

In ```modelflow``` the index series is typically understood to represent a date. 

For yearly models a list of integers like in the above example works fine.<br>

For higher frequency models (quarterly, monthly, weekly,daily, etc.) the index can be one of several pandas date types, but users are encouraged to use `pd.period_range()` to create date indexes.

:::{warning}

Not all datetypes work well with the graphics routines of modelflow.  Users are advised to use the ```pd.period_range()``` method to generate date indexes.

For example:
```   
    dates = pd.period_range(start='1975q1',end='2125q4',freq='Q')
    df.index=dates
```

:::

```{index} single: DataFrame; leads and lags in Modelflow
```
```{index} single: Leads in modelflow
```
```{index} single: Lags in modelflow
```



## Leads and lags

`Pandas` does not support the economic idea of leads and lags per se (although the `.shift()` operator can ebe used to emulate the same idea in ordered `dataframes`.

`Modelflow` explicitly supports the idea of leads and lags. In modelflow leads and lags can be indicated by following the variable with a parenthesis and either -1 or -2 two for one or two period lags (where the number following the negative sign indicates the number of time periods that are lagged). Positive numbers are used for forward leads (no +sign required).

When a method defined by the `modelflow` class encounters something like `A(-1)`, it will take the value from the row above the current row. No matter if the index is an integer, a year, quarter or a millisecond. The same goes for leads, `A(+1)` will return the value of `A` in the next row. 

As a result in a quarterly model `B=A(-4)` would assign B the value of A from the same quarter in the previous year. 

```{index} single: DataFrame; .upd() modelflow method
```

## .upd() method of modelflow


The ```.upd()``` method extends pandas by giving the user a concise and expressive way to modify data in a `dataframe` using a syntax that a database-manager or macroeconomic modeler might find more natural. 

Notably it allows the user to employ formula's to do updates, and supports both lags and leads on variables. 

```.upd()``` can be used to:

 - Perform different types of  updates
 - Perform multiple updates each on a new line 
 - Perform changes over specific periods 
 - Use one input which is used for all time frames, or a separate input for each time 
 - Preserve pre-shock growth rates for out of sample time-periods 
 - Display results 
 

```{index} single: DataFrame; operators 
```

### ```.upd()``` method operators 
Below are some of the operators that can be used in the ```.upd()``` method

**Types of update:** 

| Update to perform |Use this operator|
| :- | :---|
 Set a variable equal to the input| = 
 Add the input to the LHS variable | + 
 Set the variable to itself multiplied by the input | *
 Increase/Decrease the variable by a percent of itself (1+input/100) | %
 Set the growth rate of the variable to the input | =growth
 Change the growth rate of the variable to its current growth rate plus the input value in percentage points  | +growth 
 Specify the amount by which the variable should increase from its previous period level ($\Delta = var_t - var_{t-1}$)| =diff

:::{Note}

The syntax of an update command requires that there be a space between variable names and the operators.

Thus ```df.upd("A = 7")``` is fine, but ```df.upd("A =7")``` will generate an error.

Similarly  ```df.upd("A * 1.1")``` is fine, but ```df.upd("A* 1.1")``` will generate an error.
:::

```{index} single: DataFrame; .upd() a Modelflow extension to pandas DataFrame
```
```{index} single: DataFrame; .upd() examples
```

 ### ```.upd()``` some examples
First, create a dataframe using standard pandas syntax.  In this instance with years as the index and a dictionary defining the variables and their data.

In [24]:
# Create a dataframe using standard pandas

df = pd.DataFrame({'B': [1,1,1,1,1,1],'C':[1,2,3,6,8,9],'E':[4,4,4,4,4,4]},index=[v for v in range(2020,2026)])
                  
df

,B,C,E
2020,1,1,4
2021,1,2,4
2022,1,3,4
2023,1,6,4
2024,1,8,4
2025,1,9,4


```{index} single: DataFrame; .upd() to create new variable 
```

#### Use .upd to create a new variable (= operator)

With standard pandas a user can add a column (series) to a dataframe simply by assigning a adding to a dataframe.  For example:

```df['NEW2']=[17,12,14,15]```

```.upd()``` provides this functionality as well.

In [25]:
df2=df.upd('c = 142') 
df2

,B,C,E
2020,1,142,4
2021,1,142,4
2022,1,142,4
2023,1,142,4
2024,1,142,4
2025,1,142,4


:::{note}
The new variable name was entered as a lower case 'c' here.  Lowercase letters are not legal ```modelflow``` variable names.  The `.upd()` method knows that it is part of modelflow and knows this rule. As a result, it automatically translates lowercase entries into upper case so that the statement works.
:::

```{index} single: DataFrame; .upd() multiple lines
```
```{index} single: DataFrame; .upd() specific time periods
```

#### Multiple updates and specific time periods

The modelflow method ```.upd()``` takes a string as an argument.  That string can contain a single update command or can contain multiple commands.

Moreover by including a \<Begin End\> date clause in a given update command, the update will be restricted to the associated time period.

The below illustrates this, modifying two existing variables A, B over different time periods and creating two new variables, `C` and `D`.  

```{note}
The third line inherits the time period of the previous line.

Note also, the submitted string can include comments as well (denoted with the standard python # indicator).
```



In [26]:
df.upd("""
# Same number of values as years
<2021 2024> A = 42 44 45 46    # 4 years
<2020     > B = 200            # 1 year 
c = 500                        # Same period as previous line
<-0 -1> D = 33                   # All years 
""")

,B,C,E,A,D
2020,200,500,4,0.0,33.0
2021,1,2,4,42.0,33.0
2022,1,3,4,44.0,33.0
2023,1,6,4,45.0,33.0
2024,1,8,4,46.0,33.0
2025,1,9,4,0.0,33.0


```{index} single: DataFrame; .upd() Time scope of commands
```

:::{index} single: Boxes; Box   2. Time scope of .upd() commands
:::

:::{admonition} Box 2. Time scope of .upd() commands


The update command takes a variety of mathematical operators ```=, +, *, % =GROWTH, +GROWTH, =DIFF``` and applies them to data for the period set in the leading <>.

If the user wants to modify a series or group of series for only a specific point in time or a period of time, she can indicate the period in the command line.

 - If **one date** is specified the operation is applied to a single point in time
 - If **two dates**  are specifies the operation is applied over a period of time.

**The selected time period will persist** until re-set with a new time specification. This is useful if several variables are going to be updated for the same time period, but must be kept in mind if subsequent commands are to operate over a different time period. 

The time period can be reset to the full time-period by using the special \<-0 -1\> time period.  More generally:

 - -0 indicates the start of the `dataframe`
 - -1 indicates the end of the `dataframe` 
 
If no time is provided the `dataframe` start and end period will be used.  
:::

```{index} single: DataFrame; .upd() = operator
```



#### Setting specific datapoints to specific values 

The ```=``` operator causes the left-hand side variable to be set equal to the values following the ```=``` operator.
:::{note} 
Either: 
 - The number of data points provided must match the number of dates in the period specified, unless
 - Only one data point is provided. In this case its value is applied to all dates in the indicated period. 
:::

In the example below:

* The first line sets the variable A during the specified period to specific values
* The second line sets the variable B to 200 in 2023
* The third line inherits the time period set in the second line (2023) and sets the variable C equal to 500.



In [27]:
df.upd("""
# Same number of values as years
<2021 2024> A = 42 44 45 46    # 4 years
<2023     > B = 200            # 1 year 
c = 500
""")

,B,C,E,A
2020,1,1,4,0.0
2021,1,2,4,42.0
2022,1,3,4,44.0
2023,200,500,4,45.0
2024,1,8,4,46.0
2025,1,9,4,0.0


```{index} single: DataFrame; .upd() + operator
```



#### (+ operator): Adding  the specified  values to all values in a range

In the example below 42 is added to the pre-existing values of the variable B over the period 2022 and 2024.  

In [28]:
df.upd('''
# Or one number to all years in between start and end 
<2022 2024> B  +  42    # one value broadcast to 3 years 
''')

,B,C,E
2020,1,1,4
2021,1,2,4
2022,43,3,4
2023,43,6,4
2024,43,8,4
2025,1,9,4


```{index} single: DataFrame; .upd() * operator
```


#### (* operator): Multiplying all values in a range by the specified values

In the example below the pre-existing values of the variable A for years 2021, 2022 and 2023 are multiplied by three different numbers (42, 44 and 45 respectively).

In [29]:
df.upd('''
# Same number of values as years
<2021 2023> A *  42 44 55
''')

,B,C,E,A
2020,1,1,4,0.0
2021,1,2,4,0.0
2022,1,3,4,0.0
2023,1,6,4,0.0
2024,1,8,4,0.0
2025,1,9,4,0.0


```{index} single: DataFrame; .upd() % operator
```


#### (% operator) Increasing all  values in a range by a  specified percent amount 
In this example:
 - A is increased by 42 and 44% over the range 2021 through 2022.
 - B is increased by 10 percent in all years
 - C is a new variable, is created and set to 100 for the whole range (because the previous line set the active range to  <-0 -1>)
 - C is decreased by 12 percent over the range 2023 through 2025.

In [35]:
df.upd('''
<-0 -1> A = 1
<2021 2022 > A %  42 44   
<-0 -1> B % 10            # all rows 
C = 100                   # all rows persist 
<2023 2025> C % -12       # now only for 3 years 
''')


,B,C,E
2020,1,1,4
2021,1,2,4
2022,1,3,4
2023,1,6,4
2024,1,8,4
2025,1,9,4


```{index} single: DataFrame; .upd() =GROWTH operator
```


#### (=GROWTH operator): Set the percent growth rate to specified values

The `=GROWTH` operator sets the growth rate of the variable to the indicated level.


In [37]:
res = df.upd('''
# Same number of values as years
<-0 -1> A = 100
<2021 2022> A =GROWTH  1 5  
<2020> c = 100 
<2021 2025> c =GROWTH 2 
''')
print(f'Dataframe:\n{res}\n\nGrowth:\n{res.pct_change()*100}\n') # Explained b

Dataframe:
      B           C  E       A
2020  1  100.000000  4  100.00
2021  1  102.000000  4  101.00
2022  1  104.040000  4  106.05
2023  1  106.120800  4  100.00
2024  1  108.243216  4  100.00
2025  1  110.408080  4  100.00

Growth:
        B    C    E         A
2020  NaN  NaN  NaN       NaN
2021  0.0  2.0  0.0  1.000000
2022  0.0  2.0  0.0  5.000000
2023  0.0  2.0  0.0 -5.704856
2024  0.0  2.0  0.0  0.000000
2025  0.0  2.0  0.0  0.000000



```{index} single: DataFrame; .upd() +GROWTH operator
```

#### (+GROWTH operator): Add or subtract from the existing percent growth rate

The below example is a bit more complicated.

The first line sets the growth rate of A to 1% in all periods beginning in  2021

The second command adds 2 3 4 5 6 to the growth rates in each period after 2021, resulting in growth rates of 3,4,5,6,7.

In [38]:
res =df.upd('''
<-0 -1> A = 1
<2021 > A =GROWTH  1  # All selected years set to the same growth rate
a +growth   2  # Add to the existing growth rate these numbers  
''')
print(f'Dataframe:\n{res}\n\nGrowth:\n{res.pct_change()*100}\n')

Dataframe:
      B  C  E     A
2020  1  1  4  1.00
2021  1  2  4  1.03
2022  1  3  4  1.00
2023  1  6  4  1.00
2024  1  8  4  1.00
2025  1  9  4  1.00

Growth:
        B           C    E         A
2020  NaN         NaN  NaN       NaN
2021  0.0  100.000000  0.0  3.000000
2022  0.0   50.000000  0.0 -2.912621
2023  0.0  100.000000  0.0  0.000000
2024  0.0   33.333333  0.0  0.000000
2025  0.0   12.500000  0.0  0.000000



```{index} single: DataFrame; .upd() =diff operator
```


####  (=diff operator): Set the change in a variable to specific values  

$\Delta = var_t - var_{t-1} = some number$

Here sets the value of A in 2021 to 2 more than the value of 2020, and the 2022 value as 4 more than the **revised** value of 2021.

The second line creates a new variable "UPBY2" to the data frame and sets it equal to 100 for all periods,

The third line adds 2 to the previous periods value UPBY2.



In [41]:
df.upd('''
<-0 -1> A = 1
< 2021 2022> A =diff  2 4   # Same number of values as years
<2020 > UpBy2 = 100 # sets rows equal to the same  number for all years in between start and end 
<2021 2025> UpBy2 =diff  2  
''')


,B,C,E,A,UPBY2
2020,1,1,4,1.0,100.0
2021,1,2,4,3.0,102.0
2022,1,3,4,7.0,104.0
2023,1,6,4,1.0,106.0
2024,1,8,4,1.0,108.0
2025,1,9,4,1.0,110.0


```{index} single: DataFrame; .upd() update several variables simultaneously
```


#### Update several variable in one line 
Sometime there is a need to update several variable with the same value over the same time frame. To ease this case .update can accept several variables in one line

In [42]:
df.upd('''
<2022 2024> h i j k =      40      # earlier values are set to zero by default
<2020>      p q r s =       1000   # All values beginning in 2020 set to 1000
<2021 -1>   p q r s =growth 2      # -1 indicates the last year of dataframe
''')

,B,C,E,H,I,J,K,P,Q,R,S
2020,1,1,4,0.0,0.0,0.0,0.0,1000.000000,1000.000000,1000.000000,1000.000000
2021,1,2,4,0.0,0.0,0.0,0.0,1020.000000,1020.000000,1020.000000,1020.000000
2022,1,3,4,40.0,40.0,40.0,40.0,1040.400000,1040.400000,1040.400000,1040.400000
2023,1,6,4,40.0,40.0,40.0,40.0,1061.208000,1061.208000,1061.208000,1061.208000
2024,1,8,4,40.0,40.0,40.0,40.0,1082.432160,1082.432160,1082.432160,1082.432160
2025,1,9,4,0.0,0.0,0.0,0.0,1104.080803,1104.080803,1104.080803,1104.080803


**Recall we have not overwritten df, so the df dataframe is unchanged.**

In [43]:
df

,B,C,E
2020,1,1,4
2021,1,2,4
2022,1,3,4
2023,1,6,4
2024,1,8,4
2025,1,9,4


:::{note}
The method `.upd()` only operates on one variable.  A command like `.upd('A = B')` would not work. For these kind of functions, use `.mfcalc()` (see next section).
:::

```{index} single: DataFrame; upd() options
```

```{index} single: DataFrame; .upd() keep_growth KG, NKG  option
```



### Selected `.upd()` options

#### The keep_growth option (--kg and --nkg)

When changing data and for certain kinds of simulations, it can sometime be useful to be able to update variables but keep the growth rate in subsequent periods unchanged. In database management this is frequently done when two time-series with different levels are spliced together. When forecasting this is useful if you have updated historical data but your views on future growth rates are unchanged.

The -kg or --keep_growth option instructs modelflow to calculate the growth rate of the existing pre-change series, and then use it to preserve the pre-change growth rates of the series for the periods that were **not** changed. 


##### The default keep_growth behaviour

The ```keep_growth``` option determines how data in  the time periods after those where an update is executed are treated.  

If ```keep_growth``` is ```False``` then data in the sub-period after a change is left unchanged.

if ```keep_growth``` is set to "```True```" then the system will preserve the pre-change growth rate of the affected variable in the time period *after the change*. 

By default ```keep_growth``` is set to ```False```.

:::{note}
At the line level:
* ```keep_growth=True``` can be expressed as --kg
* ```keep_growth=False``` can be expressed as --nkg
:::

Consider the followingconcrete example. A ```dataframe``` df has two variables A and B, that each grow by 2% per period, with A initialized at a level of 100 and B at a level of 110 so that we can see each separately on a graph.



In [ ]:
df = pd.DataFrame(100,
     index=[v for v in range(2020,2025)],
       columns=['A','B']) 

df=df.upd("""<2021 -1> A =growth 2
           <2020 -1>   B = 110
          <2021 -1>    B =growth 2
          """)
# Store these variables for later use in comparisons
df['A_ORIG']=df['A']
df['B_ORIG']=df['B']
df


In [ ]:
df[['A','B']].plot()

The `.upd()` command below modifies both A and B by adding 5 to their levels in each of 2022 and 2023.  

For A, this is done with the `keep_growth` option set to `True` -- the `--kg` option in the code below.  This means that for A the growth rate after the shock period 2022-23 will be unchanged at 2 percent.

For series B the same shock is applied but with keep_growth set to `False` using the --nkg option.  

The keep_growth global variable is ignored in this instance as each line in the update is overriding it using the --kg option (`keep_growth=True`) and --nkg option (`keep_growth=False`).


In [ ]:
df=df.upd("""
            <2022 2023> A + 5 --kg
            <2022 2023> B + 5 --nkg
            """)

df[['A','B','A_ORIG','B_ORIG']].plot()
    

In the first example 'A' (the green and blue lines) the level of A is increased by 5 for two periods (2021-2022). The levels of the subsequent values are also increased because the previous growth rate (2%) is now applied to the new higher level of the data in 2022.

For the 'B' variable the same level change was input but because of the `--nkg` (equivalent to `keep_growth=False`) the periods after the change were unaffected. the shocked variable returns to its pre-shock level immediately in 2023.

Below are plots the growth rates of the two transformed series.

Here the growth in both series accelerates in 2022, by slightly less than 5 percentage points because a) the base of each is more than 100 in 2021 (because of the 2 percent growth in 2021). Substantially more in the case of  B, which was initialized at 110. In 2023 the growth rate of A returns to 2 percent, while the growth rate of B is actually negative because the level (see earlier graph) has fallen back to its original level.

In [ ]:
dfg=df[['A','B']].pct_change()*100
dfg.plot()

```{index} single: DataFrame; .upd() options
```
```{index} single: DataFrame; .upd() keep_growth KG, NKG  option
```


#### .upd(,,,keep_growth) some more examples





 Initialize a new dataframe, with some growth rate 

In [ ]:
# instantiate a new dataframe with one column 'A' with avlue 100 everywhere and index 2020-2025
dftest = pd.DataFrame(100,
       index=[v for v in range(2020,2026)], # create row index
       # equivalent to index=[2020,2021,2022,2023,2024,2025] 
       columns=['A'])                                 # create column name
dftest

In [ ]:
# Update a to have growth rate accelerationg linearly by 1 from 1 oercent to 5 percent
original = dftest.upd('<2021 2025> a =growth 1 2 3 4 5')  
print(f'Levels:\n{original}\n\nGrowth:\n{original.pct_change()*100}\n')

**Now update A in 2021 to 2023 to a new value**

Below performs the same operation twice, the first time the updated value is assigned to the `dataframe` `nkg` and the default behaviour of ```keep_growth``` is ```False```

In the second example the ```-kg``` line option is specified, telling modelflow to maintain the growth rates of the dependent variable in the periods after the update is executed.

In [ ]:
nokg = original.upd('''
<2021 2025>  a =growth 1 2 3 4 5 
<2021 2023>  a = 120  
''',lprint=0)

kg = original.upd('''
<2021 2025>  a =growth 1 2 3 4 5 
<2021 2023>  a = 120  --kg
''',lprint=0)


kg=kg.rename(columns={"A":"KG"})       #rename cols to facilitate display
nokg=nokg.rename(columns={"A":"NOKG"}) #rename cols to facilitate display
df=original.rename(columns={"A":"Orig"}) #rename cols to facilitate display

combo=pd.concat([kg,nokg,df], axis=1)
combo


print(f'Levels\n{combo}\n\nGrowth\n{combo.pct_change()*100}')

**Understanding the results**

In the first example where KG (keep_growth) **was set**, the level was set constant for three periods at 120 the rate of growth was 0 for the final two years of the set period.  But following this update, the level of A in 2023 is 120. With ```keep_Growth=True``` the KG variable growth at 2 percent per year in 2024 and 2025.

In the **--nkg** example, the levels of NOKG are the same as KG for 2020 through 2023, but because `--nkg` was selected the levels revert to their pre-shock values, which are lower than the 120 in 2023.  As a result the growth rate for NOKG is negative in 2024. The growth rate for 2024 remains 5 because neither the 2024 or 2025 data changed and therefore the 2025 the growth rate does not change. 



#### .upd() with the option keep_growth set globally

Above the line level option `--keep_growth` or `--kg` was used to keep the growth rate (or not) for a given operation.

This works because by default the option `Keep_growth` is set to false, implementing `--kg` at the line level temporarily set the keep_growth flag to  true for the specific line (and those following).

The `keep_growth` flag can also be set globally for all the lines by setting the option in the command line.

 ```keep_growth=True```.

Now as default, all lines will keep the growth rate (unless overridden at the line level with `--nkg` or `--no_keep_growth`.
 - c,d are updated in 2022 and 2023 and keep the growth rates afterwards
 - e the `--no_keep_growth` in this line prevents the updating 2024-2025

In [ ]:
# Create a data frame
dftest = pd.DataFrame(100,
       index=[v for v in range(2020,2025)], # create row index
       # equivalent to index=[2020,2021,2022,2023,2024,2025] 
       columns=['A','B','C','D','E'])                                 # create column name 
dftest

In [ ]:
dfres = dftest.upd('''
<2022 2023> c = 200 
<2022 2023> d = 300  
<2022 2023> e = 400  --no_keep_growth 
''',keep_growth=True)  # <=  Set keep_growth to True for the entirety of the command, 
                       # except for e where it is overridden by the --no_keep_growth flag
print(f'Dataframe:\n{dfres}\n\nGrowth:\n{dfres.pct_change()*100}\n')

```{index} single: DataFrame; .upd() scale option
```


#### .upd(,,scale=\<number, default=1>) Scale the updates 

When running scenarios it can be useful to sensitivity analyses of model results, to better understand how the model responds when varying the intensity of a shock.

The scale option provides a mechanism for calculating a range shocks as a proportion of the initially indicated one.

When using the scale option, scale=0 implies no change (effectively the baseline) while scale=0.5 is a scenario half of the full severity.


In the example below `.upd()` is executed three times for severity equals 0. 0.5 and 1.  If the list passed to scale (named severity in this case) had five items in it, the update would be run five times -- one time for each item in the list.  


This example just prints outputs, a more interesting example would involve the solving a model using different levels of a given shock. 


In [ ]:
print(f'input dataframe: \n{df}\n\n')
for severity in [0,0.5,1]: 
    # First make a dataframe with some growth rate 
    res = df.upd('''
    <2021 2025>
    a =growth 1 2 3 4 5 
    b + 10
    ''',scale=severity)
    print(f'{severity=}\nDataframe:\n{res}\n\nGrowth:\n{res.pct_change()*100}\n\n')
    #  
    # Here the updated dataframe is only printed. 
    # A more realistic use case is to simulate a model like this: 
    # dummy_ = mpak(res,keep='Severity {serverity}')    # more realistic 

```{index} single: DataFrame; .upd() lprint option
```


#### .upd(,,lprint=True ) prints values the before and after update  

The ```lPrint``` option of the method ```upd()``` is set to ```= False``` by default.  By setting it true, an update command will output the results of the calculation comparing the values of the dataframe (over the impacted period) before, after and the difference between the two.



In [ ]:
df.upd('''
# Same number of values as years
<2021 2022> A *  42 44
''',lprint=1);

```{index}  single: DataFrame; .upd() create option
```



#### .upd(,,create=True ) Requires the variable to exist  
Until now .upd has created variables if they did not exist in the input dataframe.

To catch misspellings the parameter ```create``` can be set to False. 
New variables will not be created, and an exception will be raised. 

Here Pythons exception handling is used, so the notebook will continue to run the cells below. 

In [ ]:
try:
    xx = df.upd('''
    # Same number of values as years
    <2021 2022> Aa *  42 44
    ''',create=False)
    print(xx)
except Exception as inst:
    xx = None
    print(inst) 
